# --- Day 25: Snowverload ---
Still somehow without snow, you go to the last place you haven't checked: the center of Snow Island, directly below the waterfall.

Here, someone has clearly been trying to fix the problem. Scattered everywhere are hundreds of weather machines, almanacs, communication modules, hoof prints, machine parts, mirrors, lenses, and so on.

Somehow, everything has been wired together into a massive snow-producing apparatus, but nothing seems to be running. You check a tiny screen on one of the communication modules: Error 2023. It doesn't say what Error 2023 means, but it does have the phone number for a support line printed on it.

"Hi, you've reached Weather Machines And So On, Inc. How can I help you?" You explain the situation.

"Error 2023, you say? Why, that's a power overload error, of course! It means you have too many components plugged in. Try unplugging some components and--" You explain that there are hundreds of components here and you're in a bit of a hurry.

"Well, let's see how bad it is; do you see a big red reset button somewhere? It should be on its own module. If you push it, it probably won't fix anything, but it'll report how overloaded things are." After a minute or two, you find the reset button; it's so big that it takes two hands just to get enough leverage to push it. Its screen then displays:
```
SYSTEM OVERLOAD!

Connected components would require
power equal to at least 100 stars!
```
"Wait, how many components did you say are plugged in? With that much equipment, you could produce snow for an entire--" You disconnect the call.

You have nowhere near that many stars - you need to find a way to disconnect at least half of the equipment here, but it's already Christmas! You only have time to disconnect three wires.

Fortunately, someone left a wiring diagram (your puzzle input) that shows how the components are connected. For example:
```
jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr
```
Each line shows the name of a component, a colon, and then a list of other components to which that component is connected. Connections aren't directional; abc: xyz and xyz: abc both represent the same configuration. Each connection between two components is represented only once, so some components might only ever appear on the left or right side of a colon.

In this example, if you disconnect the wire between hfx/pzl, the wire between bvb/cmg, and the wire between nvd/jqt, you will divide the components into two separate, disconnected groups:
9 components: cmg, frs, lhk, lsr, nvd, pzl, qnr, rsh, and rzs.
6 components: bvb, hfx, jqt, ntq, rhn, and xhk.
Multiplying the sizes of these groups together produces 54.

Find the three wires you need to disconnect in order to divide the components into two separate groups. What do you get if you multiply the sizes of these two groups together?

## --- Part Two ---
You climb over weather machines, under giant springs, and narrowly avoid a pile of pipes as you find and disconnect the three wires.

A moment after you disconnect the last wire, the big red reset button module makes a small ding noise:
```
System overload resolved!
Power required is now 50 stars.
```
Out of the corner of your eye, you notice goggles and a loose-fitting hard hat peeking at you from behind an ultra crucible. You think you see a faint glow, but before you can investigate, you hear another small ding:
```
Power required is now 49 stars.

Please supply the necessary stars and
push the button to restart the system.
```

In [1]:
from pathlib import Path
import os

yr = 2023
d = 25

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  formatted_input = {'Nodes': [], 'Edges': []}
  for l in inp.splitlines():
    s = l.split(':')[0].strip()
    ds = list(l.split(':')[1].split(' '))[1:]
    if s not in formatted_input['Nodes']:
      formatted_input['Nodes'].append(s)
    for d in ds:
      if (d, s) not in formatted_input['Edges'] and (s, d) not in formatted_input['Edges']:
        formatted_input['Edges'].append((s,d))
  for e in formatted_input['Edges']:
    for el in e:
      if el not in formatted_input['Nodes']:
        formatted_input['Nodes'].append(el)
  for e in formatted_input['Edges']:
    while tuple(reversed(e)) in formatted_input['Edges']:
      formatted_input['Edges'].remove(tuple(reversed(e)))

  assert(len(set(formatted_input['Edges'])) == len(formatted_input['Edges']))

  formatted_input['Nodes'] = list(sorted(formatted_input['Nodes']))
  formatted_input['Edges'] = list(sorted(formatted_input['Edges']))

  return formatted_input

In [3]:
def randomized_contraction(formatted_input):
  '''
  Naive way of solving using randomized edge deletion.

  Nice for small graphs but need to run iteratively in
  order to find optimal solution and probability of
  success is
  1/(n**2) where n is number of nodes.
  Not practical for 1000+ nodes but could work if you let it
  run for a week or did some uber-threading
  '''
  import copy
  import random

  G = copy.deepcopy(formatted_input)

  while len(G['Nodes']) != 2:

    cur_edge = random.choice(G['Edges'])

    # Prune the edge and any duplicates
    while cur_edge in G['Edges']:
      G['Edges'].remove(cur_edge)
    while tuple(reversed(cur_edge)) in G['Edges']:
      G['Edges'].remove(tuple(reversed(cur_edge)))

    # Any edge that contains the destination for this
    # edge will now point to the source of this edge
    to_remove = []
    for e in G['Edges']:
      if cur_edge[1] in e:
        to_remove.append(e)
        other_n = [n for n in e if n != cur_edge[1]][0]
        G['Edges'].append((cur_edge[0], other_n))
    for e in to_remove:
      G['Edges'].remove(e)

    if cur_edge[1] not in G['Nodes']:
      print(cur_edge[1])
    assert(cur_edge[1] in G['Nodes'])
    G['Nodes'].remove(cur_edge[1])
  return G

In [4]:
def spectral_partition(formatted_input):
  '''
  Implementation of spectral graph
  partitioning algorithm based on the work
  of Miroslav Fiedler
  '''
  import numpy as np
  import copy

  def incidence_matrix(G):
    arr = np.zeros((len(G['Nodes']), len(G['Edges'])))
    for i, n in enumerate(G['Nodes']):
      for j, e in enumerate(G['Edges']):
        if n in e:
          arr[i,j] = 1 if n==e[0] else -1
    return arr

  G = copy.deepcopy(formatted_input)

  incidence = incidence_matrix(G)
  laplacian = np.matmul(incidence, incidence.T)

  eigs = np.linalg.eig(laplacian)

  # Get the Eigenvector Corresponding to the 2nd Largest Eigenvalue
  eig_vals = eigs[1][:,np.argsort(eigs[0])[1]]

  labels = {n: int(ev > 0) for n, ev in zip(G['Nodes'], eig_vals)}

  # Find edges that bridge labeled sections
  cut_edges = [e for e in G['Edges'] if labels[e[0]] + labels[e[1]]==1]

  return cut_edges, labels


def product_of_groups(formatted_input):
  import numpy as np
  _, labels = spectral_partition(formatted_input)
  label_vals = np.array(list(labels.values()))
  return np.multiply(np.sum(label_vals==0), np.sum(label_vals==1))

In [5]:
import time

t = time.time()

formatted_input = format_input(inp)

print(product_of_groups(formatted_input))

print("\nRUNTIME: ", time.time()-t)

548960

RUNTIME:  1.6834969520568848
